In [10]:
import os
import shutil
import zipfile

In [11]:
base_folder = os.getcwd()
temporary_folder = os.path.join(os.getcwd(), "tmp")

In [12]:
def unzip_files():
# Unzip file on a temporary folder
    if os.path.exists(temporary_folder):
        shutil.rmtree(temporary_folder)
        
    if not os.path.exists(temporary_folder):
        os.makedirs(temporary_folder)
        
    local_file_name = os.path.join(base_folder, "training_dataset", "trainingandtestdata.zip")
    with zipfile.ZipFile(local_file_name, 'r') as zip_ref:
        zip_ref.extractall(temporary_folder)

In [29]:
def cleansing_and_tokenizing(tweet):
# Cleansing and tokenizing tweet
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords 
    from string import punctuation 
    from bs4 import BeautifulSoup
    import re
    
    terms_to_remove = set(stopwords.words("english") + ["USERTAGGING","URL"])
    tweet = BeautifulSoup(tweet, 'html.parser').get_text() # Extracts text from HTML (just in case!)
    tweet = tweet.lower() # Converts text to lower-case
    tweet = re.sub("((www\.[^\s]+)|(https?://[^\s]+))", "URL", tweet) # Replces URLs by URL constan
    tweet = re.sub("@[^\s]+", "USERTAGGING", tweet) # Replaces usernames by USERTAGGING constant 
    tweet = re.sub(r"#([^\s]+)", r"\1", tweet) # Removes the # in #hashtag
    for p in punctuation: 
        tweet = tweet.replace(p, "") # Removes punctiation
    tweet = word_tokenize(tweet) # Creates a list of words
    words = ""
    for each_word in tweet:
        if each_word not in terms_to_remove:
            words = words + " " + each_word
    # return [word for word in tweet if word not in terms_to_remove]
    return words[1:]

In [13]:
unzip_files()

In [18]:
from pyspark.sql import SparkSession, functions


spark = SparkSession.builder.master("local").appName("Training Twitter Sentiment Analysis").getOrCreate()
training_data = spark.read.load(
    "tmp/training.1600000.processed.noemoticon.csv",
    format="csv")
training_data = training_data.withColumnRenamed("_c0", "label") \
    .withColumnRenamed("_c1", "tweet_id") \
    .withColumnRenamed("_c2", "date") \
    .withColumnRenamed("_c3", "query") \
    .withColumnRenamed("_c4", "user") \
    .withColumnRenamed("_c5", "tweet")


In [19]:
sample_size = 10000
training_data = training_data.sample(sample_size / 1600000)

training_data = training_data.select(functions.col("label"), functions.col("tweet"))

In [30]:
udf_cleansing_and_tokenizing = functions.udf(cleansing_and_tokenizing)
training_data = training_data.withColumn("tweet_cleansed", udf_cleansing_and_tokenizing(functions.col("tweet")))

In [32]:
training_data.show(5)

+-----+--------------------+--------------------+
|label|               tweet|      tweet_cleansed|
+-----+--------------------+--------------------+
|    0|Poor Joshy is sic...|poor joshy sick d...|
|    0|@nicolerichie OH ...|         oh yes miss|
|    0|Just heard that t...|heard found sandr...|
|    0|I don't like the ...|dont like previou...|
|    0|worked his heart ...|worked heart toda...|
+-----+--------------------+--------------------+
only showing top 5 rows



In [33]:
training_data.describe()

DataFrame[summary: string, label: string, tweet: string, tweet_cleansed: string]

In [39]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="tweet_cleansed", outputCol="words")
training_data = tokenizer.transform(training_data)

In [40]:
from pyspark.ml.feature import Word2Vec

word2vec = Word2Vec(inputCol="words", outputCol="vectorized") 
model = word2vec.fit(training_data)
training_data = model.transform(training_data)   


In [41]:
training_data.show(5)

+-----+--------------------+--------------------+--------------------+--------------------+
|label|               tweet|      tweet_cleansed|               words|          vectorized|
+-----+--------------------+--------------------+--------------------+--------------------+
|    0|Poor Joshy is sic...|poor joshy sick d...|[poor, joshy, sic...|[-0.0268824268132...|
|    0|@nicolerichie OH ...|         oh yes miss|     [oh, yes, miss]|[-0.0248483438044...|
|    0|Just heard that t...|heard found sandr...|[heard, found, sa...|[-0.0023246163701...|
|    0|I don't like the ...|dont like previou...|[dont, like, prev...|[-0.0151059520430...|
|    0|worked his heart ...|worked heart toda...|[worked, heart, t...|[-0.0098882078018...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [47]:
training_data = training_data.withColumn("label", functions.col("label").cast("integer"))
training, test = training_data.select("label", "vectorized").randomSplit([0.5, 0.5])

In [48]:
from pyspark.ml.classification import NaiveBayes
model = NaiveBayes(featuresCol="vectorized", labelCol="label")
model_train = model.fit(training)
predictions = model_train.transform(test)
predictions.show(5)



Py4JJavaError: An error occurred while calling o407.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 21) (192.168.19.2 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(NaiveBayes$$Lambda$3543/0x00000008014d9040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.ScalaUDF.eval(ScalaUDF.scala:1193)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.update(Summarizer.scala:374)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.update(Summarizer.scala:344)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.update(interfaces.scala:563)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator$$anonfun$1.$anonfun$applyOrElse$2(Aggreg

In [ ]:
    model = NaiveBayes(featuresCol="features", labelCol="label")
    model_train = model.fit(training)
    predictions = model_train.transform(test)
    predictions.show(5)

    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    print(accuracy)